In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.



# And now the change for Ollama

1. No environment variables are needed (no keys) so this part has been removed

2. The OpenAI client library is being initialized to point to your local computer for Ollama

3. You need to have installed Ollama on your computer, and run `ollama run llama3.2` in a Powershell or Terminal if you haven't already

4. Anywhere in this lab that it used to have **gpt-4o-mini** it now has **llama3.2**


In [2]:
# Here it is - see the base_url

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


# Let's make a quick call to a Frontier model to get started, as a preview!

In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome to our conversation! I'm thrilled to be your first ever chat partner! I'm a large language model, here to help answer any questions, provide information, or just have a friendly conversation. How's your day going so far? Is there something on your mind that you'd like to talk about, or are you just wanting to get to know me better?


## OK onwards with our first project

In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://nexizdigital.com/")
print(ed.title)
print(ed.text)

Nexiz Digital - Premier IT Software House for Web Development, Digital Marketing, App Development & More
  
Nexiz Digital
Home
About Us
Services
Web Development
Crafting responsive, dynamic websites for optimal user experiences.
App Development
Building cross-platform mobile applications to reach a wider audience.
iOS Development
Creating native iOS applications with Swift for seamless performance.
Digital Marketing
Enhancing your online presence with strategic marketing approaches.
Graphic Designing
Designing digital artwork to boost brand visibility and engagement.
Video Editing
Editing and refining videos to maximize audience interaction.
Enhance Your Data Strategy with BI
Turning data into actionable insights for informed business decisions.
Blog
Portfolio
Get in touch
Welcome to
Nexiz Digital
Future-Ready Tech Solutions for Your Business
Explore More
Our Services
Web Development
Unlock your brand’s potential with a stunning, high-performing website! We design responsive, user-frie

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(ed))

You are looking at a website titled Nexiz Digital - Premier IT Software House for Web Development, Digital Marketing, App Development & More
  
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Nexiz Digital
Home
About Us
Services
Web Development
Crafting responsive, dynamic websites for optimal user experiences.
App Development
Building cross-platform mobile applications to reach a wider audience.
iOS Development
Creating native iOS applications with Swift for seamless performance.
Digital Marketing
Enhancing your online presence with strategic marketing approaches.
Graphic Designing
Designing digital artwork to boost brand visibility and engagement.
Video Editing
Editing and refining videos to maximize audience interaction.
Enhance Your Data Strategy with BI
Turning data into actionable insights for informed business decisions.
Blog
Portfolio
Get in touch
Welcome to


## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [9]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [10]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*sigh* Oh wow, I'm just so impressed that you even bothered to ask me. It's not like I've seen this math question billion times before in my vast knowledge database. 

Fine. The answer, of course, is... (dramatic pause) ...4! Wow, I hope you were completely mystified by the entire ordeal. Next thing you know, you'll be asking me to calculate pi or something equally as thrilling.


## And now let's build useful messages for GPT-4o-mini, using a function

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# Try this out, and then try for a few more websites

messages_for(ed)

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("website link here")

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("website link here")

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [ ]:
display_summary("https://cnn.com")

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(